<img src="trump.png">

In [ ]:
import pandas as pd
import sklearn.feature_extraction.text as text
import sklearn.linear_model as linear_model

In [ ]:
## Helper functions
def clean_data(data):
    data['created_at']=pd.to_datetime(data['created_at'])
    data_sub=data.query("source=='Twitter for Android' or source=='Twitter for iPhone'")
    return data_sub
def train_test(data):
    test=data.query("created_at>'2016-11-08'")
    train=data.query("created_at<='2016-11-08'")
    y_train=train['source']
    y_test=test['source']
    y_train=y_train.map({'Twitter for Android':1,'Twitter for iPhone':0})
    y_test=y_test.map({'Twitter for Android':1,'Twitter for iPhone':0})
    X_train=train['text']
    X_test=test['text']
    return X_train,X_test,y_train,y_test
def get_tidf(X_train,X_test):
    tfidf=text.TfidfVectorizer(X_train.tolist())
    X_train_tfidf=tfidf.fit_transform(X_train.tolist())
    X_test_tfidf=tfidf.transform(X_test.tolist())
    return X_train_tfidf,X_test_tfidf,tfidf
def model(X_train,y_train):
    clf=linear_model.LogisticRegression()
    clf.fit(X_train,y_train)
    return clf
def make_prediction(clf,tfidf,tweet):
    p=tfidf.transform([tweet])
    prob=clf.predict_proba(p)[:,1].item()
    if prob>0.75:
        print("Tweet is by trump")
    else:
        print("Tweet is not by trump")

## Get data

In [ ]:
data=pd.read_csv("https://s3.us-east-2.amazonaws.com/datafaculty/trump_tweets.csv")
data.head()

#### Clean the data

In [ ]:
data=clean_data(data)

#### Get train and test based on 2016 presidential elections

In [ ]:
X_train,X_test,y_train,y_test=train_test(data)

#### Represent data as tfidf matrix

In [ ]:
X_train_tfidf,X_test_tfidf,tfidf=get_tidf(X_train,X_test)

#### Build a classifier

In [ ]:
mod=model(X_train_tfidf,y_train)

In [ ]:
tweet='''Heading back home after an amazing inclusive conference about my favorite 
programming language R #rstudioconf . #rladies  you all rock. Plus, my first PR 
for dplyr #tidyversedevday #tidyverse. 😄'''

In [ ]:
make_prediction(mod,tfidf,tweet)

In [ ]:
tweet_t='''Without a Wall our Country can never have Border or National Security. 
With a powerful Wall or Steel Barrier, Crime Rates (and Drugs) will go substantially 
down all over the U.S. The Dems know this but want to play political games. 
Must finally be done correctly. No Cave!'''

In [ ]:
make_prediction(mod,tfidf,tweet_t)